In [71]:
#check if visually selected units are also significant
#not used in hte final report

In [ ]:
import os
import glob
import time 
import copy
import pickle
import random
import sklearn
import warnings
import scipy
import numpy as np
import pandas as pd
import statsmodels
import matplotlib.pyplot as plt

import utils.helper_func as hf
import utils.plotting as plotting
import utils.decoding as decoding

warnings.filterwarnings("ignore")

In [2]:
patient_id = 46
session_nr = 1

#top_dir = '/media/alado/TOSHIBA_EXT/thesis/output'
top_dir = '/home/anastasia/epiphyte/anastasia/output'
folder = f'08-sign_lfp_{patient_id}'
data_folder = f'05-Hilbert_transform_{patient_id}'
filtering_type = ['theta', 'slow_gamma', 'fast_gamma']

fs = 32768
#The resulting sample rate is up / down times the original sample rate.
up = 1
down=32
fs_downs = (up/down)*fs
dt = 1/fs_downs      # sampling period/time/interval or time resolution, often denoted as T

times = np.linspace(-500, 1000, num = 1501)
time_zero_idx = np.where(times == hf.find_nearest(times, 0))[0][0]

df_patient_info = pd.read_csv(f'{top_dir}/{patient_id}_channel_info.csv')
dict_replace = {'LA1':'LAA1', 'LA2':'LAA2', 'LA3':'LAA3', 'LA4':'LAA4', 'LA5':'LAA5', 'LA6':'LAA6', 'LA7':'LAA7', 'LA8':'LAA8',
               'RA1':'RAA1', 'RA2':'RAA2', 'RA3':'RAA3', 'RA4':'RAA4', 'RA5':'RAA5', 'RA6':'RAA6', 'RA7':'RAA7', 'RA8':'RAA8',}
df_patient_info_new = df_patient_info.replace(dict_replace)
df_stim_info = pd.read_csv(f'{top_dir}/{patient_id}_df_stim_info.csv')
all_stim = np.unique(df_stim_info['stim_id'])
all_stim_name = np.unique(df_stim_info['stim_name'])
all_brain_areas = np.unique(df_patient_info['recording_site'])
#brain_areas = ['LAA', 'LAH', 'LEC','LMH', 'LPHC', 'RAA', 'RAH', 'REC', 'RMH', 'RPHC']
brain_areas = np.unique(df_patient_info['brain_area'])

n_channels = 80
n_stim_present = 10
n_stim = 42
all_stimuli = np.linspace(0, n_stim-1, n_stim, dtype=int)

In [3]:
time_chunks = ['200_500', '300_600', '200_600', '-500_50']
time_chunks_dict = {'200_500_st':700, '200_500_end':1000, '300_600_st':800, '300_600_end':1100, 
                    '200_600_st':700, '200_600_end':1100, '-500_50_st':0, '-500_50_end':450}
n_pca_comp = 10
columns_data_array = []
for j in filtering_type:
    for i in range(len(df_patient_info['channel_name'])):
        ch = df_patient_info.loc[i,'channel_name']
        ch_site = df_patient_info.loc[i,'recording_site']
        
        for tm in time_chunks:
            columns_data_array.append(f'{j}_{ch}_{tm}')
            
        for comp in range(n_pca_comp):
            columns_data_array.append(f'{j}_{ch}_pca_{comp+1}')

In [4]:
if os.path.isfile(f'{top_dir}/{folder}/df_data_post.csv'):
    df_data_pre = pd.read_csv(f'{top_dir}/{folder}/df_data_pre.csv')
    df_data_post = pd.read_csv(f'{top_dir}/{folder}/df_data_post.csv')
    df_stimuli = pd.read_csv(f'{top_dir}/{folder}/df_stimuli.csv')
else:
    n_pca_components = 10
    df_data_pre, _ = decoding.create_data_df(df_stim_info, df_patient_info, all_stim, filtering_type, time_chunks, time_chunks_dict, columns_data_array, data_folder, n_pca_components, 'pre', pca_mode=True)
    df_data_post, df_stimuli = decoding.create_data_df(df_stim_info, df_patient_info, all_stim, filtering_type, time_chunks, time_chunks_dict, columns_data_array, data_folder, n_pca_components, 'post', pca_mode=True)
    df_data_post.to_csv(f'{top_dir}/{folder}/df_data_post.csv')
    df_data_pre.to_csv(f'{top_dir}/{folder}/df_data_pre.csv')
    df_stimuli.to_csv(f'{top_dir}/{folder}/df_stimuli.csv')

In [24]:
df_pvals = pd.DataFrame(all_stimuli, columns=['stim_index'])
for filt in filtering_type:
    for time_period in time_chunks[:-1]:       
        for i in range(len(df_patient_info['channel_name'])):
            ch = df_patient_info.loc[i,'channel_name']
            cols_to_use_pre = [filt+'_'+ch+'_'+'-500_50']
            cols_to_use_post = [filt+'_'+ch+'_'+time_period]
            values_pre = df_data_pre[df_data_pre.columns & cols_to_use_pre]
            values_post = df_data_post[df_data_post.columns & cols_to_use_post]
            df_stimuli['values_pre'] = values_pre.iloc[:,0].values
            df_stimuli['values_post'] = values_post.iloc[:,0].values
            all_pvals = []
            for st in all_stimuli:
                array_pre = df_stimuli.loc[df_stimuli.stim_index == st, 'values_pre']
                array_post = df_stimuli.loc[df_stimuli.stim_index == st, 'values_post']
                a = scipy.stats.wilcoxon(array_pre, array_post)[1]
                all_pvals.append(a)

            df_pvals[filt+'_'+ch+'_'+time_period] = all_pvals
            df_stimuli = df_stimuli.drop(columns=['values_pre', 'values_post'])
df_pvals.to_csv(f'{top_dir}/{folder}/df_pvals.csv')      

In [25]:
for filt in filtering_type:
    counts_per_filt = []
    for time_period in time_chunks[:-1]:
        for i in range(len(df_patient_info['channel_name'])):
            ch = df_patient_info.loc[i,'channel_name']
            count = df_pvals[(df_pvals[filt+'_'+ch+'_'+time_period]<0.01)].count()
            #print(filt+'_'+ch+'_'+time_period)
            #print(count[0])
            counts_per_filt.append(count[0])
        print(filt+'_'+time_period)
        print(sum(counts_per_filt))

theta_200_500
31
theta_300_600
55
theta_200_600
77
slow_gamma_200_500
24
slow_gamma_300_600
106
slow_gamma_200_600
150
fast_gamma_200_500
64
fast_gamma_300_600
142
fast_gamma_200_600
216


In [26]:
df_pvals_corr = pd.DataFrame(all_stimuli, columns=['stim_index'])
for name, values in df_pvals.iteritems():
    if name != 'stim_index':
        #print(name)
        result = statsmodels.stats.multitest.multipletests(values, alpha=0.05, method='fdr_bh')
        df_pvals_corr[name] = result[1]
        df_pvals_corr[name+'_reject'] = result[0]

In [27]:
for filt in filtering_type:
    counts_per_filt = []
    for time_period in time_chunks[:-1]:
        for i in range(len(df_patient_info['channel_name'])):
            ch = df_patient_info.loc[i,'channel_name']
            count = df_pvals_corr[(df_pvals_corr[filt+'_'+ch+'_'+time_period]<0.05)].count()
            #print(filt+'_'+ch+'_'+time_period)
            #print(count[0])
            counts_per_filt.append(count[0])
        print(filt+'_'+time_period)
        print(sum(counts_per_filt))

theta_200_500
0
theta_300_600
0
theta_200_600
0
slow_gamma_200_500
0
slow_gamma_300_600
6
slow_gamma_200_600
6
fast_gamma_200_500
10
fast_gamma_300_600
14
fast_gamma_200_600
20


In [19]:
for filt in filtering_type:
    counts_per_filt = []
    for time_period in time_chunks[:-1]:
        for i in range(len(df_patient_info['channel_name'])):
            ch = df_patient_info.loc[i,'channel_name']
            print(filt+'_'+ch+'_'+time_period)
            print(df_pvals.loc[df_pvals[filt+'_'+ch+'_'+time_period]<0.05, 'stim_index'].iloc[0])
            

theta_CSC1_200_500
41
theta_CSC2_200_500


IndexError: single positional indexer is out-of-bounds

In [20]:
filt+'_'+ch+'_'+time_period

'theta_CSC2_200_500'

In [29]:
interesting_units_53 = [4, 6, 24, 31]
interesting_units_46 = [1, 7, 8, 16, 18, 26, 29, 31, 33, 37, 42, 45, 46, 59, 67, 68, 69, 88]



In [105]:
session_nr = 1
patient_id = 46
position, stim_id, filename, stim_name, is_500_days, paradigm, time = get_screening_data(patient_id, session_nr)
stim_frame = cp.make_dataframe(position, stim_id, filename, stim_name, is_500_days, paradigm, time)

stim_data = {}
stim_nums = stim_frame.stim_num.unique()

for stim_num in stim_nums:
    # setting up to get the stimulus image
    meta = stim_frame.loc[stim_frame.stim_num == stim_num, :].iloc[0]
    stim_fname = meta["filename"]
    stim_data[stim_fname] = (stim_num,
                            meta["stim_name"],
                            meta["paradigm"],
                            mpl.imread(os.path.join(PATH_TO_IMAGES, stim_fname))) ## read image file into an array


img_order = IMAGE_ORDER_ALL
img_order = np.concatenate(img_order, axis=0)

data_df = []

for unit in interesting_units_46:
    spikes = get_spiking_activity(patient_id, session_nr, unit)
    stim_index, eventtimes = get_scr_eventtimes(patient_id, session_nr)
    region = get_brain_region(patient_id, unit)
    unit_type = get_unit_type(patient_id, session_nr, unit)

    unit_pvals = get_scr_stats_as_df(patient_id, session_nr, unit)
    
    all_hists = []
    times = spikes
    save_unit = False
    pvals = []
    
    for i_row, img_fname in enumerate(img_order):
        #for i_col, img_fname in enumerate(img):
        stim_num, stim_name, paradigm, image = stim_data[img_fname]

        event_data = stim_frame.loc[stim_frame.stim_num == stim_num, ['position', 'time']]
        sp_times = times
        sep = .2
        hist_max = 0
        time_data = []
        pre_spikes = []
        post_spikes = []
        color_data = []
        hist_data = defaultdict(list)

        for name, color in zip(POSITIONS, COLORS):
            #print(name)
            events = event_data.loc[event_data.position == name, 'time'].values
            pre_spikes = []
            post_spikes = []
            #print(events.shape)
            #print(sp_times)
            for event in events:
                idx_sp = (sp_times >= event - T_PRE) & (sp_times <= event + T_POST)
                #print(idx_sp)
                if idx_sp.any():
                    temp = sp_times[idx_sp] - event
                    #print(temp)
                    pre = np.sum(np.where((-500 < temp) & (temp < 0)))
                    post = np.sum(np.where((170 < temp) & (temp < 600)))
                    time_data.append(temp)
                    pre_spikes.append(pre)
                    post_spikes.append(post)
                    t_hist, _ = np.histogram(temp, HIST_BINS)

                    hist_data[name].append(t_hist * 1000 / HIST_BIN_WIDTH)
                else:
                    time_data.append([-2*T_PRE])   # hack of eventplot

                color_data.append(color)
            try:
                p_val = scipy.stats.wilcoxon(pre_spikes, post_spikes)[1]
            except ValueError:
                p_val = 1
            #print(p_val)
            if name == 'post':
                pvals.append(p_val)
    pvals = statsmodels.stats.multitest.multipletests(pvals, method='fdr_bh')[1]
    data_df.append(list(pvals))

df = pd.DataFrame.from_records(data_df)
df = df.transpose()
df.columns = interesting_units_46
df.index = img_order

In [106]:
df

,1,7,8,16,18,26,29,31,33,37,42,45,46,59,67,68,69,88
summer1.jpg,0.397698,0.862030,1.0,0.174383,1.0,0.009115,1.0,0.013672,0.068297,0.110306,0.009115,0.032171,0.057977,0.268337,0.307169,0.200329,0.286452,0.324411
summer2.jpg,0.880396,0.847807,1.0,0.300230,1.0,0.027344,1.0,0.020175,0.020508,0.171591,0.024858,0.024127,0.048468,0.083447,0.444235,0.164619,0.286452,0.183203
summer3.jpg,0.599292,0.452293,1.0,0.122595,1.0,0.069411,1.0,0.013672,0.492188,0.134722,0.009115,0.018930,0.041016,0.895332,0.756258,0.196799,0.457000,0.253784
summer4.jpg,0.503195,0.568436,1.0,0.122595,1.0,0.019531,1.0,0.058812,0.031901,0.053268,0.014673,0.011719,0.048468,0.764706,0.222940,1.000000,0.460754,0.085449
summer5.jpg,1.000000,0.452293,1.0,0.829484,1.0,0.011719,1.0,0.058812,0.037287,0.189700,0.009115,0.011719,0.102539,0.436581,0.129195,0.800789,0.286452,0.035093
name-summer.jpg,0.397698,0.454804,1.0,0.163731,1.0,0.017934,1.0,0.041016,0.068297,0.485760,0.014673,0.011719,0.184371,0.083447,0.129195,0.444235,0.475966,0.035093
guy1.jpg,1.000000,0.847807,1.0,0.122595,1.0,0.020508,1.0,0.168164,0.024659,0.134722,0.014673,0.018930,0.707849,0.436581,0.756258,0.732505,0.475966,0.045573
tom1.jpg,0.943748,0.862030,1.0,0.122595,1.0,0.009115,1.0,0.024966,0.160129,0.053268,0.063889,0.024127,0.041016,0.436581,0.093998,0.126930,0.344398,0.075628
tom2.jpg,0.880396,0.452293,1.0,0.122595,1.0,0.020508,1.0,0.040766,0.045937,0.166334,0.017434,0.128035,0.481316,0.097250,0.204360,0.126930,0.767525,0.016406
tom3.jpg,0.599292,0.452293,1.0,0.129957,1.0,0.027344,1.0,0.047483,0.332523,0.134495,0.090445,0.058297,0.599616,0.184971,0.759868,0.127312,0.500182,0.032280
